In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 147.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [11]:
# !pip install -U ultralytics pyyaml
from pathlib import Path
import zipfile, yaml

ZIP_PATH = Path("/content/Chess Pieces.v24-416x416_aug.yolov11.zip")  # your zip
ROOT = Path("/content/roboflow_chess_full")                           # where to extract
ROOT.mkdir(parents=True, exist_ok=True)

# --- extract ---
with zipfile.ZipFile(ZIP_PATH, "r") as zf:
    zf.extractall(ROOT)

print("Extracted to:", ROOT)

# --- find data.yaml (Roboflow export places it near the split folders) ---
data_yaml = next(ROOT.rglob("data.yaml"), None)
assert data_yaml is not None, "data.yaml not found inside the ZIP contents."

# --- rewrite train/val/test to absolute paths (prevents path issues) ---
with open(data_yaml, "r") as f:
    cfg = yaml.safe_load(f)

base = data_yaml.parent
def _abs(p):
    return str((base / p).resolve())

for k in ("train", "val", "test"):
    if k in cfg and cfg[k]:
        cfg[k] = _abs(cfg[k])

# ensure 'nc' aligns with 'names'
if "names" in cfg and "nc" not in cfg:
    cfg["nc"] = len(cfg["names"]) if isinstance(cfg["names"], list) else len(cfg["names"].keys())

with open(data_yaml, "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Using data.yaml:", data_yaml)
print("train:", cfg.get("train"))
print("val:", cfg.get("val"))
print("test:", cfg.get("test"))


Extracted to: /content/roboflow_chess_full
Using data.yaml: /content/roboflow_chess_full/data.yaml
train: /content/train/images
val: /content/valid/images
test: /content/test/images


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.6 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO
import torch, platform
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch: 2.8.0+cu126 | CUDA: True


In [7]:
# choose a size for your GPU: n/s/m/l/x  (start with n or s)
model = YOLO("yolo11s.pt")

results = model.train(
    data=str(data_yaml),
    epochs=100,
    imgsz=512,         # lower to 512/448 if VRAM is tight
    batch=16,          # drop to 8/4 if you see CUDA OOM
    device=0,
    pretrained=True,
)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/roboflow_chess_full/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru

In [14]:
metrics = model.val(data=str(data_yaml), imgsz=640, device=0)
print(metrics)

# best weights path
from pathlib import Path
best = sorted(Path("runs/detect").rglob("weights/best.pt"))[-1]
print("Best weights:", best)

# run predictions on the val set (or any folder/file)
model = YOLO(str(best))
model.predict(source="/content/roboflow_chess_full/valid/images", imgsz=640, conf=0.25, device=0)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 497.8±175.9 MB/s, size: 21.4 KB)
val: Scanning /content/roboflow_chess_full/valid/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 58/58 81.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.2it/s 3.3s
                   all         58        386      0.983      0.993      0.988      0.823
          black-bishop         13         22      0.998      0.955      0.968      0.763
            black-king         29         29      0.993          1      0.995      0.871
          black-knight         26         30      0.997          1      0.995      0.804
            black-pawn         25         77      0.998          1      0.995      0.808
           black-queen         11         11      0.965          1      0.995      0.848
            black-rook         24 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bishop', 1: 'black-bishop', 2: 'black-king', 3: 'black-knight', 4: 'black-pawn', 5: 'black-queen', 6: 'black-rook', 7: 'white-bishop', 8: 'white-king', 9: 'white-knight', 10: 'white-pawn', 11: 'white-queen', 12: 'white-rook'}
 obb: None
 orig_img: array([[[223, 225, 225],
         [223, 225, 225],
         [221, 223, 223],
         ...,
         [218, 220, 220],
         [218, 220, 220],
         [219, 221, 221]],
 
        [[223, 225, 225],
         [222, 224, 224],
         [221, 223, 223],
         ...,
         [218, 220, 220],
         [218, 220, 220],
         [219, 221, 221]],
 
        [[223, 225, 225],
         [222, 224, 224],
         [221, 223, 223],
         ...,
         [218, 220, 220],
         [218, 220, 220],
         [219, 221, 221]],
 
        ...,
 
        [[197, 202, 201],
         [203, 208, 207],
         [208,

In [15]:
from google.colab import files
import shutil

# Replace this with your folder path:
folder_path = "/content/runs/detect/val4"
# folder_path = "/content/runs/detect/train2"

# Create a zip file
shutil.make_archive("download_me", "zip", folder_path)

# Download it
files.download("download_me.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>